In [1]:
import pandas as pd
import io
import requests

In [2]:
urls = [
    "https://declarations.cnil.fr/declarations-a-b.csv",
    "https://declarations.cnil.fr/declarations-c.csv",
    "https://declarations.cnil.fr/declarations-d-f.csv",
    "https://declarations.cnil.fr/declarations-g-k.csv",
    "https://declarations.cnil.fr/declarations-l-m.csv",
    "https://declarations.cnil.fr/declarations-n-r.csv",
    "https://declarations.cnil.fr/declarations-s-z.csv",
    "https://declarations.cnil.fr/declarations-autres.csv"
]

In [3]:
csvs = [pd.read_csv(url, skiprows=[0], sep=';', encoding='ISO-8859-1') for url in urls]

/Users/arachez/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
csvs[5] = pd.read_csv(urls[5], sep=';', skiprows=[0], header=None, names=csvs[0].columns, encoding='ISO-8859-1')

/Users/arachez/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
assert(len(set(name for index in [csv.columns for csv in csvs] for name in index)) == 13)

In [6]:
eta = pd.read_csv("https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true", sep=';')

In [7]:
dem = pd.concat(csvs, ignore_index=True)

In [8]:
dem.dropna(subset=['Organisme déclarant adresse', 'Organisme déclarant code postal'], inplace=True)
dem['address'] = dem['Organisme déclarant adresse'].astype(str).apply(lambda x: x.lower())
# dem['cp'] = pd.to_numeric(dem['Organisme déclarant code postal'], errors='coerse')
dem['cp'] = dem['Organisme déclarant code postal']

In [9]:
eta.dropna(subset=['Adresse', 'Code postal'], inplace=True)
eta['address'] = eta['Adresse'].astype(str).apply(lambda x: x.lower())
# eta['cp'] = pd.to_numeric(eta['Code postal'], errors='coerse')
eta['cp'] = eta['Code postal'].astype(int).astype(str)

In [10]:
keep_eta = ['address', 'cp', 'Coordonnee X', 'Coordonnee Y', 'Latitude', 'Longitude']
m = pd.merge(eta[keep_eta], dem, on=['address', 'cp'], how='inner')

In [39]:
m.head()

,address,cp,Coordonnee X,Coordonnee Y,Latitude,Longitude,Organisme déclarant raison sociale,Organisme déclarant nom du service,Organisme déclarant adresse,Organisme déclarant code postal,Organisme déclarant ville,Organisme déclarant siren,Service chargé du droit d'accès,Date d'enregistrement,Finalité du fichier déclaré,Catégories et destinataires des données,Numéro de déclaration,"Modification déclarée (date, motif)",Type de déclaration
0,2 place georges brassens,31700,568981.5,6284678.1,43.649022,1.376363,ECOLE ÉLÉMENTAIRE WEIDKNNET,NaN,2 PLACE GEORGES BRASSENS,31700,BLAGNAC,100700004,NaN,13-01-2018,"NS58: Affaires scolaires, périscolaires, extra...",NaN,2141373 - v0,NaN,Déclaration simplifiée
1,chemin de cantelauze,31470,557286.1,6274157.7,43.552127,1.234389,COLLEGE CANTELAUZE,GESTION,CHEMIN DE CANTELAUZE,31470,FONSORBES,193123387,NaN,31-05-2008,AU9: Biométrie : accès aux cantines scolaires,NaN,1297067 - v0,NaN,Déclaration simplifiée
2,chemin de cantelauze,31470,557160.6,6274114.0,43.551709,1.232849,COLLEGE CANTELAUZE,GESTION,CHEMIN DE CANTELAUZE,31470,FONSORBES,193123387,NaN,31-05-2008,AU9: Biométrie : accès aux cantines scolaires,NaN,1297067 - v0,NaN,Déclaration simplifiée
3,chemin de cantelauze,31470,557288.5,6274135.7,43.551929,1.234425,COLLEGE CANTELAUZE,GESTION,CHEMIN DE CANTELAUZE,31470,FONSORBES,193123387,NaN,31-05-2008,AU9: Biométrie : accès aux cantines scolaires,NaN,1297067 - v0,NaN,Déclaration simplifiée
4,village,31460,605240.7,6274891.2,43.566752,1.827424,MAIRIE DE CAMBIAC,NaN,VILLAGE,31460,CAMBIAC,213101025,MAIRIE DE CAMBIAC,03-05-2001,NS38: Liste électorale des ressortissants d'un...,NaN,754209 - v0,NaN,Déclaration simplifiée


In [47]:
m['siren'] = m['Organisme déclarant siren'].astype(str)

In [68]:
print(eta.shape)
print(dem.shape)
print(m.shape)

(60717, 26)
(1562796, 15)
(29248, 20)


In [69]:
m.head()

,address,cp,Coordonnee X,Coordonnee Y,Latitude,Longitude,Organisme déclarant raison sociale,Organisme déclarant nom du service,Organisme déclarant adresse,Organisme déclarant code postal,Organisme déclarant ville,Organisme déclarant siren,Service chargé du droit d'accès,Date d'enregistrement,Finalité du fichier déclaré,Catégories et destinataires des données,Numéro de déclaration,"Modification déclarée (date, motif)",Type de déclaration,siren
0,2 place georges brassens,31700,568981.5,6284678.1,43.649022,1.376363,ECOLE ÉLÉMENTAIRE WEIDKNNET,NaN,2 PLACE GEORGES BRASSENS,31700,BLAGNAC,100700004,NaN,13-01-2018,"NS58: Affaires scolaires, périscolaires, extra...",NaN,2141373 - v0,NaN,Déclaration simplifiée,100700004
1,chemin de cantelauze,31470,557286.1,6274157.7,43.552127,1.234389,COLLEGE CANTELAUZE,GESTION,CHEMIN DE CANTELAUZE,31470,FONSORBES,193123387,NaN,31-05-2008,AU9: Biométrie : accès aux cantines scolaires,NaN,1297067 - v0,NaN,Déclaration simplifiée,193123387
2,chemin de cantelauze,31470,557160.6,6274114.0,43.551709,1.232849,COLLEGE CANTELAUZE,GESTION,CHEMIN DE CANTELAUZE,31470,FONSORBES,193123387,NaN,31-05-2008,AU9: Biométrie : accès aux cantines scolaires,NaN,1297067 - v0,NaN,Déclaration simplifiée,193123387
3,chemin de cantelauze,31470,557288.5,6274135.7,43.551929,1.234425,COLLEGE CANTELAUZE,GESTION,CHEMIN DE CANTELAUZE,31470,FONSORBES,193123387,NaN,31-05-2008,AU9: Biométrie : accès aux cantines scolaires,NaN,1297067 - v0,NaN,Déclaration simplifiée,193123387
4,village,31460,605240.7,6274891.2,43.566752,1.827424,MAIRIE DE CAMBIAC,NaN,VILLAGE,31460,CAMBIAC,213101025,MAIRIE DE CAMBIAC,03-05-2001,NS38: Liste électorale des ressortissants d'un...,NaN,754209 - v0,NaN,Déclaration simplifiée,213101025


In [102]:
cols = ['address', 'cp', 'Coordonnee X', 'Coordonnee Y', 'Latitude', 'Longitude', 'Organisme déclarant raison sociale', 'Organisme déclarant siren', 'siren']
by = ['Organisme déclarant siren']

def my_agg(g):
    names = {
        'siren': g['siren'].min(),
        'siren count': len(g['siren']),
        'address': g['address'].astype(str).min(),
        'cp': g['cp'].min(),
        'Coordonnee X': g['Coordonnee X'].min(),
        'Coordonnee Y': g['Coordonnee Y'].min(),
        'Latitude': g['Latitude'].min(),
        'Longitude': g['Longitude'].min()
    }
    return pd.Series(names, index=names.keys())

In [124]:
r = m[cols].groupby(by=by).aggregate({
    'siren': {
        'siren': min,
        'count': len,
    },
    'Organisme déclarant raison sociale': {'nom': min},
    'address': {'address': min},
    'cp': {'cp': min},
    'Coordonnee X': {'Coordonnee X': min},
    'Coordonnee Y': {'Coordonnee Y': min},
    'Latitude': {'Latitude': min},
    'Longitude':{'Longitude': min}
})

/Users/arachez/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [125]:
r.columns = r.columns.get_level_values(1)
r.head()

,siren,count,nom,address,cp,Coordonnee X,Coordonnee Y,Latitude,Longitude
Organisme déclarant siren,,,,,,,,,
100700004.0,100700004.0,2,RECAPE CECILE,21 avenue frizac,31190,570279.2,6245451.7,43.296421,1.402275
101300002.0,101300002.0,1,RODRIGUE BLON,5 rue des vaureitres,49170,421549.9,6703786.3,47.375826,-0.691487
101600005.0,101600005.0,1,RPI 113,9 rue de saint omer,62960,651379.7,7049400.8,50.539943,2.315209
101900009.0,101900009.0,1,PROFESSEUR DES ÉCOLES,570 rue beer sheva,69009,839238.5,6522542.1,45.788275,4.792369
102300001.0,102300001.0,2,RPC HUCHENNEVILLE,rue de rosières,80132,613832.5,6966688.4,49.798777,1.797987


In [126]:
r.to_csv('geo.csv', index=False)

In [77]:
m.columns

Index(['address', 'cp', 'Coordonnee X', 'Coordonnee Y', 'Latitude',
       'Longitude', 'Organisme déclarant raison sociale',
       'Organisme déclarant nom du service', 'Organisme déclarant adresse',
       'Organisme déclarant code postal', 'Organisme déclarant ville',
       'Organisme déclarant siren', 'Service chargé du droit d'accès',
       'Date d'enregistrement', 'Finalité du fichier déclaré',
       'Catégories et destinataires des données', 'Numéro de déclaration',
       'Modification déclarée (date, motif)', 'Type de déclaration', 'siren'],
      dtype='object')

<function __main__.my_agg>